# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 9 2022 12:25PM,246061,10,8505771,Beach Products Inc,Released
1,Sep 9 2022 12:23PM,246059,10,SO94378,"Senseonics, Incorporated",Released
2,Sep 9 2022 12:21PM,246057,19,ACG-2102491188,ACG North America LLC,Released
3,Sep 9 2022 12:21PM,246056,10,0085970149,ISDIN Corporation,Released
4,Sep 9 2022 12:21PM,246056,10,0085970150,ISDIN Corporation,Released
5,Sep 9 2022 12:21PM,246056,10,0085970191,ISDIN Corporation,Released
6,Sep 9 2022 12:21PM,246056,10,0085970207,ISDIN Corporation,Released
7,Sep 9 2022 12:21PM,246056,10,0085970891,ISDIN Corporation,Released
8,Sep 9 2022 12:21PM,246056,10,0085971608,ISDIN Corporation,Released
9,Sep 9 2022 12:21PM,246056,10,0085971611,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,246055,Released,8
29,246056,Released,29
30,246057,Released,1
31,246059,Released,1
32,246061,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246055,NaN,NaN,8.0
246056,NaN,NaN,29.0
246057,NaN,NaN,1.0
246059,NaN,NaN,1.0
246061,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245651,0.0,0.0,1.0
245733,0.0,1.0,0.0
245763,0.0,1.0,0.0
245771,0.0,0.0,1.0
245975,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245651,0,0,1
245733,0,1,0
245763,0,1,0
245771,0,0,1
245975,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245651,0,0,1
1,245733,0,1,0
2,245763,0,1,0
3,245771,0,0,1
4,245975,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245651,,,1
1,245733,,1,
2,245763,,1,
3,245771,,,1
4,245975,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 9 2022 12:25PM,246061,10,Beach Products Inc
1,Sep 9 2022 12:23PM,246059,10,"Senseonics, Incorporated"
2,Sep 9 2022 12:21PM,246057,19,ACG North America LLC
3,Sep 9 2022 12:21PM,246056,10,ISDIN Corporation
32,Sep 9 2022 12:08PM,246055,10,ISDIN Corporation
40,Sep 9 2022 11:56AM,246053,20,"Exact-Rx, Inc."
41,Sep 9 2022 11:53AM,246052,22,"NBTY Global, Inc."
42,Sep 9 2022 11:17AM,246047,10,"Methapharm, Inc."
56,Sep 9 2022 11:06AM,246046,10,"Methapharm, Inc."
57,Sep 9 2022 11:03AM,246044,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 9 2022 12:25PM,246061,10,Beach Products Inc,,,1
1,Sep 9 2022 12:23PM,246059,10,"Senseonics, Incorporated",,,1
2,Sep 9 2022 12:21PM,246057,19,ACG North America LLC,,,1
3,Sep 9 2022 12:21PM,246056,10,ISDIN Corporation,,,29
4,Sep 9 2022 12:08PM,246055,10,ISDIN Corporation,,,8
5,Sep 9 2022 11:56AM,246053,20,"Exact-Rx, Inc.",,,1
6,Sep 9 2022 11:53AM,246052,22,"NBTY Global, Inc.",,,1
7,Sep 9 2022 11:17AM,246047,10,"Methapharm, Inc.",,,14
8,Sep 9 2022 11:06AM,246046,10,"Methapharm, Inc.",,1,
9,Sep 9 2022 11:03AM,246044,10,ISDIN Corporation,,,12


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 9 2022 12:25PM,246061,10,Beach Products Inc,1,,
1,Sep 9 2022 12:23PM,246059,10,"Senseonics, Incorporated",1,,
2,Sep 9 2022 12:21PM,246057,19,ACG North America LLC,1,,
3,Sep 9 2022 12:21PM,246056,10,ISDIN Corporation,29,,
4,Sep 9 2022 12:08PM,246055,10,ISDIN Corporation,8,,
5,Sep 9 2022 11:56AM,246053,20,"Exact-Rx, Inc.",1,,
6,Sep 9 2022 11:53AM,246052,22,"NBTY Global, Inc.",1,,
7,Sep 9 2022 11:17AM,246047,10,"Methapharm, Inc.",14,,
8,Sep 9 2022 11:06AM,246046,10,"Methapharm, Inc.",,1,
9,Sep 9 2022 11:03AM,246044,10,ISDIN Corporation,12,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 9 2022 12:25PM,246061,10,Beach Products Inc,1,,
1,Sep 9 2022 12:23PM,246059,10,"Senseonics, Incorporated",1,,
2,Sep 9 2022 12:21PM,246057,19,ACG North America LLC,1,,
3,Sep 9 2022 12:21PM,246056,10,ISDIN Corporation,29,,
4,Sep 9 2022 12:08PM,246055,10,ISDIN Corporation,8,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 9 2022 12:25PM,246061,10,Beach Products Inc,1.0,NaN,NaN
1,Sep 9 2022 12:23PM,246059,10,"Senseonics, Incorporated",1.0,NaN,NaN
2,Sep 9 2022 12:21PM,246057,19,ACG North America LLC,1.0,NaN,NaN
3,Sep 9 2022 12:21PM,246056,10,ISDIN Corporation,29.0,NaN,NaN
4,Sep 9 2022 12:08PM,246055,10,ISDIN Corporation,8.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 9 2022 12:25PM,246061,10,Beach Products Inc,1.0,0.0,0.0
1,Sep 9 2022 12:23PM,246059,10,"Senseonics, Incorporated",1.0,0.0,0.0
2,Sep 9 2022 12:21PM,246057,19,ACG North America LLC,1.0,0.0,0.0
3,Sep 9 2022 12:21PM,246056,10,ISDIN Corporation,29.0,0.0,0.0
4,Sep 9 2022 12:08PM,246055,10,ISDIN Corporation,8.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3444581,85.0,9.0,2.0
15,492039,21.0,0.0,5.0
16,984019,6.0,4.0,0.0
19,246057,1.0,0.0,0.0
20,492076,17.0,12.0,0.0
21,982918,2.0,2.0,0.0
22,246052,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3444581,85.0,9.0,2.0
1,15,492039,21.0,0.0,5.0
2,16,984019,6.0,4.0,0.0
3,19,246057,1.0,0.0,0.0
4,20,492076,17.0,12.0,0.0
5,21,982918,2.0,2.0,0.0
6,22,246052,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,85.0,9.0,2.0
1,15,21.0,0.0,5.0
2,16,6.0,4.0,0.0
3,19,1.0,0.0,0.0
4,20,17.0,12.0,0.0
5,21,2.0,2.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,85.0
1,15,Released,21.0
2,16,Released,6.0
3,19,Released,1.0
4,20,Released,17.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21,22
Status,,,,,,,
Completed,2.0,5.0,0.0,0.0,0.0,0.0,0.0
Executing,9.0,0.0,4.0,0.0,12.0,2.0,0.0
Released,85.0,21.0,6.0,1.0,17.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21,22
0,Completed,2.0,5.0,0.0,0.0,0.0,0.0,0.0
1,Executing,9.0,0.0,4.0,0.0,12.0,2.0,0.0
2,Released,85.0,21.0,6.0,1.0,17.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21,22
0,Completed,2.0,5.0,0.0,0.0,0.0,0.0,0.0
1,Executing,9.0,0.0,4.0,0.0,12.0,2.0,0.0
2,Released,85.0,21.0,6.0,1.0,17.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()